- Main KITTI readme (old): https://github.com/yanii/kitti-pcl/blob/master/KITTI_README.TXT  
- About Intrinsics: http://ksimek.github.io/2013/08/13/intrinsic/
- Car scheme: http://www.cvlibs.net/publications/Geiger2013IJRR.pdf
- https://avisingh599.github.io/vision/visual-odometry-full/
- Image with reprojection: https://yadi.sk/i/JAIIsbP5dHAELg
- https://github.com/cgarg92/Stereo-visual-odometry
- http://www.cs.toronto.edu/~urtasun/courses/CSC2541/03_odometry.pdf
- https://dsp.stackexchange.com/questions/2736/step-by-step-camera-pose-estimation-for-visual-tracking-and-planar-markers

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import cv2

import sys
sys.path.append('..')

from shared.data import KITTIData,  VisualOdometry, draw_matches, draw_keypoints

%matplotlib widget

import matplotlib.pyplot as plt
import matplotlib as mpl
print(mpl.style.available)
mpl.style.use('grayscale')


from mpl_toolkits.mplot3d import Axes3D
# https://github.com/matplotlib/ipympl

In [ ]:
DATASET_DIR = os.path.join('../', 'data/KITTI/dataset')
dataset = KITTIData(DATASET_DIR)

In [ ]:
frame_idx = 105
c_l_img, c_r_img = dataset.get_images(frame_idx)
n_l_img, n_r_img = dataset.get_images(frame_idx+1)
Q_left = dataset.get_left_Q_matrix()
gt_transform = dataset._get_transform_mtrx(frame_idx)
gt_pose_transform = dataset.get_poses_transform(frame_idx)
c_pose = dataset.get_poses()[frame_idx]
C_left, _ = dataset.get_С_matrix()

print(gt_transform)
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.imshow(c_l_img)
ax2.imshow(n_l_img)

In [ ]:
vo = VisualOdometry()
c_depth_frame = vo.process_depth(c_l_img, c_r_img, Q_left)
n_depth_frame = vo.process_depth(n_l_img, n_r_img, Q_left)

fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.imshow(c_depth_frame[:,:,2])
ax2.imshow(n_depth_frame[:,:,2])

In [ ]:
c_feats, n_feats = vo.get_features(c_l_img, n_l_img)

c_img_canvas = c_l_img.copy()
n_img_canvas = n_l_img.copy()
draw_keypoints(c_img_canvas, n_img_canvas, c_feats, n_feats)
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.imshow(c_img_canvas)
ax2.imshow(n_img_canvas)

In [ ]:
def show_3d_features(c_pnts3d, n_pnts3d, C_left):
    P_left = np.hstack((C_left, np.transpose([[0, 0, 0]])))

    c_pnts_2d = vo.reproject_3d_to_2d(c_pnts3d, P_left)
    n_pnts_2d = vo.reproject_3d_to_2d(n_pnts3d, P_left)

    c_img_canvas = c_l_img.copy()
    n_img_canvas = n_l_img.copy()
    # Check after clique and projection
    draw_keypoints(c_img_canvas, n_img_canvas, c_pnts_2d, n_pnts_2d)
    # # Rendering valid features
    fig, (ax1, ax2) = plt.subplots(2, 1)
    ax1.imshow(c_img_canvas)
    ax2.imshow(n_img_canvas)

In [ ]:
### Get 3D points 
c_pnts_3d, c_ft_idxs = vo.reproject_2d_to_3d_points(c_feats, c_depth_frame)
n_pnts_3d, n_ft_idxs = vo.reproject_2d_to_3d_points(n_feats, n_depth_frame)

ft_idxs = c_ft_idxs & n_ft_idxs

c_pnts_3d = c_pnts_3d[ft_idxs]
n_pnts_3d = n_pnts_3d[ft_idxs]

# c_img_canvas = c_l_img.copy()
# n_img_canvas = n_l_img.copy()
# draw_keypoints(c_img_canvas, n_img_canvas, c_feats[ft_idxs], n_feats[ft_idxs])
# fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
# ax1.imshow(c_img_canvas)
# ax2.imshow(n_img_canvas)

transform = vo.get_transform(c_pnts_3d, n_pnts_3d, C_left, type_='PnPRansac')
# print(transform[:3,:3], transform[:3,3])
# print(gt_pose_transform[:3,:3], gt_pose_transform[:3,3])

n_pose = vo.get_next_pose(transform, c_pose)

pred_pose_transform = n_pose @ np.linalg.inv(c_pose)
# print(pred_pose_transform[:3,:3], pred_pose_transform[:3,3])
err = np.abs(gt_pose_transform-pred_pose_transform)
print(err[:3,:3])
print(err[:3,3])

show_3d_features(c_pnts_3d, n_pnts_3d, C_left)

In [ ]:
### Filter 
cl_idxs, _ = vo.max_clique_filter(c_pnts_3d, n_pnts_3d)
c_pnts_3d_cl = c_pnts_3d[cl_idxs]
n_pnts_3d_cl = n_pnts_3d[cl_idxs]

# c_img_canvas = c_l_img.copy()
# n_img_canvas = n_l_img.copy()
# draw_keypoints(c_img_canvas, n_img_canvas, c_feats[cl_idxs], n_feats[cl_idxs])
# fig, (ax1, ax2) = plt.subplots(2, 1, figsize=[15,9])
# ax1.imshow(c_img_canvas)
# ax2.imshow(n_img_canvas)

transform = vo.get_transform(c_pnts_3d_cl, n_pnts_3d_cl, C_left, type_='PnP')
# print(transform[:3,:3], transform[:3,3])
# print(gt_pose_transform[:3,:3], gt_pose_transform[:3,3])

n_pose = vo.get_next_pose(transform, c_pose)
# print(n_pose[:3,:3])
# print(n_pose[:3,3])

pred_pose_transform = n_pose @ np.linalg.inv(c_pose)
# print(pred_pose_transform[:3,:3], pred_pose_transform[:3,3])
err = np.abs(gt_pose_transform-pred_pose_transform)
print(err[:3,:3])
print(err[:3,3])

show_3d_features(c_pnts_3d_cl, n_pnts_3d_cl, C_left)